In [1]:
import main as m
import time
import osmnx as ox
import importlib as i
from shapely import wkt
import folium
from folium.plugins import MarkerCluster

#isnull

In [2]:
from folium import plugins


In [3]:
m = folium.Map([41.97, 2.81])

plugins.LocateControl().add_to(m)

# If you want get the user device position after load the map, set auto_start=True
plugins.LocateControl(auto_start=True).add_to(m)

m

In [3]:
PATH = {}
PATH["rawDATA"] = "data/unprocessed/"
t = "tourism"
keys = ["attraction", "artwork", "gallery", "museum", "viewpoint", "yes", "memorial", "monument"]

# https://wiki.openstreetmap.org/wiki/Key:historic
# t = "historic"
# keys = ["memorial", "monument"]
# in case of timeout problem set it:
ox.settings.timeout=600
# memorial gives timeout https://wiki.openstreetmap.org/wiki/Key:historic
#keys = ["monument"]
# timeout? https://github.com/gboeing/osmnx/pull/492

In [6]:
# check
place = "Mallorca"
for k in keys:
    searchTerm = str("'"+t+"': '"+k+"'")
    print(str("'"+t+"': '"+k+"'"))
    start = time.time()
    tags = {searchTerm}
    #print({t:k})
    raw = ox.geometries_from_place(place, tags={t:k})
    #display(raw.explore())
    try:
        raw.to_parquet(PATH["rawDATA"]+place+"-"+k+".parquet")
    except:
        raw.to_csv(PATH["rawDATA"]+place+"-"+k+".csv")  
    end = time.time()
    print(end - start)

'tourism': 'memorial'
2.5565288066864014
'tourism': 'monument'
1.4308702945709229


In [7]:
keys

['memorial', 'monument']

In [4]:
# read in 
locations = {}
place = "Mallorca"
for k in keys:
    print(k)
    try:
        locations[k] = m.gpd.read_parquet(PATH["rawDATA"]+place+"-"+k+".parquet")
    except:
        locations[k] = m.pd.read_csv(PATH["rawDATA"]+place+"-"+k+".csv", low_memory=False)
        locations[k]["geometry"] = locations[k]["geometry"].apply(wkt.loads)
        locations[k] = m.gpd.GeoDataFrame(locations[k], geometry = locations[k]["geometry"], crs = 'EPSG:4326')
        #t = m.gpd.GeoDataFrame(locations[k], geometry = m.gpd.points_from_xy(locations[k]['x'], locations[k]['y']), crs = 'EPSG:4326')
        #locations[k].crs = "epsg:4326"
        #t = m.pd.read_csv(PATH["rawDATA"]+k+".csv")
        #locations[k] = m.gpd.GeoDataFrame(t, geometry=t.geometry, crs = 'EPSG:4326')

attraction
artwork
gallery
museum
viewpoint
yes
memorial
monument


In [5]:
colors = [
    'red',
    'blue',
    'gray',
    'darkred',
    'lightred',
    'orange',
    'beige',
    'green',
    'darkgreen',
    'lightgreen',
    'darkblue',
    'lightblue',
    'purple',
    'darkpurple',
    'pink',
    'cadetblue',
    'lightgray',
    'black'
]
from folium.plugins import Search

icons = ["magnet", "hammer", "palette", "building-columns","eye","question"]

icons = ["magnet", "hammer", "palette", "building-columns","eye","question", "monument", "monument"]

# icons list
# https://fontawesome.com/search?q=caravan&o=r

ma = folium.Map(location=[47.1611615, 19.5057541], zoom_start=7, tiles='OpenStreetMap')
plugins.LocateControl().add_to(ma)
plugins.LocateControl(auto_start=True).add_to(ma)

search = Search(
    layer=None,
    geom_type='Point',
    placeholder='Search for a location',
    collapsed=False,
    search_label='name',
    weight=3,
    position='topright'
).add_to(ma)

marker_cluster = MarkerCluster().add_to(ma)
c = 0
l = list(locations.keys())
WHERE = "Mallorca"
for i in l:
    print(i)
    locations[i]["lat"] = locations[i].geometry.centroid.y
    locations[i]["lon"] = locations[i].geometry.centroid.x
    try:
        locations[i]["websiteBool"] = locations[i]["website"].apply(lambda x: 1 if m.pd.isnull(x) == True else 0)
    except:
        continue
    #print(locations[i].columns.values.tolist())
    for _, r in locations[i].iterrows():
        #m.folium.Marker(location=[r['lat'], r['lon']], popup='length: {} <br> area: {}'.format(r['Shape_Leng'], r['Shape_Area'])).add_to(m)
        #folium.Marker(location=[r['lat'], r['lon']], popup=r["name"], icon=folium.Icon(color=colors[c], prefix='fa', icon=icons[c])).add_to(ma)
        #link = "<a href=https://www.google.com/search?q=" + r["name"] + " target=_blank >" + r["name"] + "</a>"
        #linkGog = str(WHERE+" "+str(r["name"])+" "+str(r["closestCity"]))
        #link = linkk.replace(" ", "+")
        link = str("<a href=https://www.google.com/search?q=" + str(r["name"]) + " target=_blank >" + str(r["name"]) + "</a>")
        link = linkk.replace(" ", "+")
        #print(geometryLink)
        if r["name"] == None:
            linkName = "Link"
        else:
            linkName = r["name"]
        
        try:
            if r["websiteBool"] == 0: # if it is 0, means there is a website
            #print(r["website"])
                link = str("<a href="+str(r["website"])+" target=_blank > "+ str(linkName)+ "</a>")
            else:
            #print("wrong link", r["website"])
                link = str("<a href=https://www.google.com/search?q=" + str(r["name"]) + " target=_blank >" + str(r["name"]) + "</a>")
        except:
            link = str("<a href=https://www.google.com/search?q=" + str(r["name"]) + " target=_blank >" + str(r["name"]) + "</a>")
            
        #link = str("<a href=https://www.google.com/search?q="+WHERE+" "+str(r["name"])+" target=_blank > "+str(r["name"])+ "</a>")
        print(link)
        folium.Marker(location=[r['lat'], r['lon']], 
                      popup=link, 
                      icon=folium.Icon(color=colors[c], prefix='fa', icon=icons[c])).add_to(marker_cluster)
    c += 1
#ma
ma.save("Mallorca2.html")

AssertionError: Search can only index FeatureGroup, MarkerCluster, GeoJson, and TopoJson layers at this time.

In [8]:
colors = [
    'red',
    'blue',
    'gray',
    'darkred',
    'lightred',
    'orange',
    'beige',
    'green',
    'darkgreen',
    'lightgreen',
    'darkblue',
    'lightblue',
    'purple',
    'darkpurple',
    'pink',
    'cadetblue',
    'lightgray',
    'black'
]

icons = ["magnet", "hammer", "palette", "building-columns","eye","question"]

icons = ["magnet", "hammer", "palette", "building-columns","eye","question", "monument", "monument"]

# icons list
# https://fontawesome.com/search?q=caravan&o=r

ma = folium.Map(location=[47.1611615, 19.5057541], zoom_start=7, tiles='OpenStreetMap')
plugins.LocateControl().add_to(ma)
plugins.LocateControl(auto_start=True).add_to(ma)

plugins.Geocoder().add_to(ma)


marker_cluster = MarkerCluster().add_to(ma)
c = 0
l = list(locations.keys())
WHERE = "Mallorca"
for i in l:
    print(i)
    locations[i]["lat"] = locations[i].geometry.centroid.y
    locations[i]["lon"] = locations[i].geometry.centroid.x
    #print(locations[i].columns.values.tolist())
    for _, r in locations[i].iterrows():
        #m.folium.Marker(location=[r['lat'], r['lon']], popup='length: {} <br> area: {}'.format(r['Shape_Leng'], r['Shape_Area'])).add_to(m)
        #folium.Marker(location=[r['lat'], r['lon']], popup=r["name"], icon=folium.Icon(color=colors[c], prefix='fa', icon=icons[c])).add_to(ma)
        #link = "<a href=https://www.google.com/search?q=" + r["name"] + " target=_blank >" + r["name"] + "</a>"
        #linkGog = str(WHERE+" "+str(r["name"])+" "+str(r["closestCity"]))
        #link = linkk.replace(" ", "+")
        name = r["name"]
        try:
            name = name.replace(" ","+")
        except:
            name = name
        try:
            link = str("<a href="+str(r["website"])+" target=_blank > "+ str(r["website"])+ "</a>")
        except:
            link = str("<a href=https://www.google.com/search?q=" + str(name) + " target=_blank >" + str(name) + "</a>")
        #link = link.replace(" ", "+")
        print(link)
        #link = str("<a href=https://www.google.com/search?q="+WHERE+" "+str(r["name"])+" target=_blank > "+str(r["name"])+ "</a>")
        #print(link)
        folium.Marker(location=[r['lat'], r['lon']], 
                      popup=link, 
                      icon=folium.Icon(color=colors[c], prefix='fa', icon=icons[c])).add_to(marker_cluster)
    c += 1
#ma
ma.save("Mallorca.html")

attraction
<a href=https://www.cuevasdeldrach.com target=_blank > https://www.cuevasdeldrach.com</a>
<a href=nan target=_blank > nan</a>
<a href=nan target=_blank > nan</a>
<a href=nan target=_blank > nan</a>
<a href=https://cuevasdelshams.com target=_blank > https://cuevasdelshams.com</a>
<a href=http://www.santueri.org target=_blank > http://www.santueri.org</a>
<a href=nan target=_blank > nan</a>
<a href=nan target=_blank > nan</a>
<a href=http://www.mallorcaballoons.com target=_blank > http://www.mallorcaballoons.com</a>
<a href=http://nerea.cat target=_blank > http://nerea.cat</a>
<a href=nan target=_blank > nan</a>
<a href=nan target=_blank > nan</a>
<a href=nan target=_blank > nan</a>
<a href=nan target=_blank > nan</a>
<a href=http://www.buggy4fun.es target=_blank > http://www.buggy4fun.es</a>
<a href=nan target=_blank > nan</a>
<a href=nan target=_blank > nan</a>
<a href=http://www.botanicactus.com target=_blank > http://www.botanicactus.com</a>
<a href=nan target=_blank > nan

C:\Users\MUSIC\AppData\Local\Temp\ipykernel_1304\2729868187.py:42: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  locations[i]["lat"] = locations[i].geometry.centroid.y
C:\Users\MUSIC\AppData\Local\Temp\ipykernel_1304\2729868187.py:43: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  locations[i]["lon"] = locations[i].geometry.centroid.x
C:\Users\MUSIC\AppData\Local\Temp\ipykernel_1304\2729868187.py:42: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  locations[i]["lat"] = locations[i].geometry.centroid.y
C:\Users\MUSIC\AppData\Local\Temp\ipykernel_1304\2729868187.py:43: UserWa

<a href=https://www.google.com/search?q=None target=_blank >None</a>
<a href=https://www.google.com/search?q=None target=_blank >None</a>
<a href=https://www.google.com/search?q=None target=_blank >None</a>
<a href=https://www.google.com/search?q=None target=_blank >None</a>
<a href=https://www.google.com/search?q=None target=_blank >None</a>
<a href=https://www.google.com/search?q=None target=_blank >None</a>
<a href=https://www.google.com/search?q=None target=_blank >None</a>
<a href=https://www.google.com/search?q=None target=_blank >None</a>
<a href=https://www.google.com/search?q=mirador+de+la+Cova+Foradada target=_blank >mirador+de+la+Cova+Foradada</a>
<a href=https://www.google.com/search?q=None target=_blank >None</a>
<a href=https://www.google.com/search?q=Mirador+des+Pontàs target=_blank >Mirador+des+Pontàs</a>
<a href=https://www.google.com/search?q=None target=_blank >None</a>
<a href=https://www.google.com/search?q=Punta+des+Pagell target=_blank >Punta+des+Pagell</a>
<a hr

C:\Users\MUSIC\AppData\Local\Temp\ipykernel_1304\2729868187.py:43: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  locations[i]["lon"] = locations[i].geometry.centroid.x


In [44]:
ma.save("Hungary.html")

In [16]:
t = str(locations['museum'].geometry[0])

In [17]:
t

'POINT (18.2269083 46.0789939)'

In [8]:
ma.save("Hungary.html")

In [58]:
import pandas as pd

In [7]:
import folium
from folium.plugins import Search

# Create a map centered at a specific location
ma = folium.Map(location=[45.523, -122.675], zoom_start=12)

# Add the Search plugin
search = Search(
    layer=None,
    geom_type='Point',
    placeholder='Search for a location',
    collapsed=False,
    search_label='name',
    weight=3,
    position='topright'
).add_to(ma)

# Add some markers to the map
folium.Marker(location=[45.523, -122.675], popup='Portland, OR').add_to(ma)
folium.Marker(location=[45.504, -122.635], popup='Milwaukie, OR').add_to(ma)
folium.Marker(location=[45.523, -122.655], popup='Sellwood, OR').add_to(ma)

# Display the map
ma

AssertionError: Search can only index FeatureGroup, MarkerCluster, GeoJson, and TopoJson layers at this time.